In [2]:
! pip install elasticdeform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 KB 2.8 MB/s eta 0:00:00


In [1]:
import numpy as np
import elasticdeform

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# example
X = np.zeros((40, 40, 40))
X[::5, ::5, ::5] = 1

# apply deformation with a random 3 x 3 grid
X_deformed = elasticdeform.deform_random_grid(X, sigma=25, points=3)

In [3]:
# load cingulate crop
skeletons = np.load('/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Rskeleton.npy')

In [12]:
skel = skeletons[0,:,:,:,0]
skel = skel!=0
print(skel.shape)
print(np.sum(skel!=0))
skel_name ='sub-1000021'

(16, 37, 37)
377


In [13]:
for sigma in [1,1.5,2]:
    for points in [11]:
        skel_elastic = elasticdeform.deform_random_grid(skel, sigma=sigma, points=points, order=0)
        IOU = np.sum(np.logical_and(skel, skel_elastic)) / np.sum(np.logical_or(skel, skel_elastic))
        print(sigma, points, np.sum(skel_elastic), IOU)

1 11 401 0.18960244648318042
1.5 11 357 0.12061068702290076
2 11 326 0.06515151515151515


In [14]:
from soma import aims

In [16]:
bucket_files = []
for iter in range(5):
    for sigma in [1,1.2,1.5,2]:
        for points in [11]:
            skel_elastic = elasticdeform.deform_random_grid(skel, sigma=sigma, points=points, order=0)
            IOU = np.sum(np.logical_and(skel, skel_elastic)) / np.sum(np.logical_or(skel, skel_elastic))
            print(sigma, points, np.sum(skel_elastic), IOU)
            arr_to_write = skel_elastic.astype(np.int16)
            arr_to_write = np.array(arr_to_write, order='F')
            vol = aims.Volume(arr_to_write)
            vol.header()['voxel_size'] = [2, 2, 2]
            filename = f'/volatile/jl277509/data/elastic_illustration/{skel_name}_sigma{sigma}_points{points}_example{iter}.nii.gz'
            aims.write(vol, filename)
            bucket_files.append(filename)

1 11 323 0.19863013698630136
1.2 11 377 0.13213213213213212
1.5 11 362 0.0915805022156573
2 11 350 0.07227138643067847
1 11 373 0.18110236220472442
1.2 11 406 0.1774436090225564
1.5 11 455 0.11081441922563418
2 11 460 0.09126466753585398
1 11 394 0.17889908256880735
1.2 11 410 0.1607669616519174
1.5 11 370 0.10666666666666667
2 11 370 0.08104196816208394
1 11 384 0.18351477449455678
1.2 11 349 0.18048780487804877
1.5 11 395 0.13196480938416422
2 11 322 0.055891238670694864
1 11 390 0.1873065015479876
1.2 11 363 0.14021571648690292
1.5 11 339 0.12401883830455258
2 11 386 0.08380681818181818


In [13]:
nb_columns = 4


import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

a = ana.Anatomist()

from soma import aims

def to_bucket(obj):
    if obj.type() == obj.BUCKET:
        return obj
    avol = a.toAimsObject(obj)
    c = aims.Converter(intype=avol, outtype=aims.BucketMap_VOID)
    abck = c(avol)
    bck = a.toAObject(abck)
    bck.releaseAppRef()
    return bck

def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()

#####################################
# We create the blocks
#####################################

block = a.createWindowsBlock(nb_columns) # nb of columns
d = {}

for i, file in enumerate(bucket_files):
    d[f'bck_{i}'] = to_bucket(a.loadObject(file))
    d[f'w_{i}'] = a.createWindow('3D', block=block)#geometry=[100+400*(i%3), 100+440*(i//3), 400, 400])
    d[f'w_{i}'].addObjects(d[f'bck_{i}'])

global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module volumepalettes
loading module paletteViewer
loading module modelGraphs
loading module profilewindow
loading module bsa_proba
loading module meshsplit
loading module selection
loading module anacontrolmenu
loading module gltf_io
loading module ana_image_math
loading module palettecontrols
loading module statsplotwindow
loading module foldsplit
loading module histogram
loading module save_resampled
loading module simple_controls
loading module valuesplotwindow
loading module gradientpalette
all python modules loaded
Anatomist started.


: 